In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors
import cv2

***Show the Image***

In [12]:
def show_image_plt(img_rgb):
    plt.imshow(img_rgb)
    plt.show()


def show_image(img):
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

***height (rows), width(cols)***

In [13]:
def resize_to_square(vegi_bgr):
    img_size = 256
    height, width = vegi_bgr.shape[:2]
    a1 = width / height
    a2 = height / width

    if (a1 > a2):
        r_img = cv2.resize(vegi_bgr, (round(img_size * a1), img_size), interpolation = cv2.INTER_AREA)
        margin = int(r_img.shape[1]/6)
        resized_img = r_img[0:img_size, margin:(margin+img_size)]

    elif(a1 < a2):
        # if height greater than width
        r_img = cv2.resize(vegi_bgr, (img_size, round(img_size * a2)), interpolation = cv2.INTER_AREA)
        margin = int(r_img.shape[0]/6)
        resized_img = r_img[margin:(margin+img_size), 0:img_size]

    elif(a1 == a2):
        # if height and width are equal
        r_img = cv2.resize(vegi_bgr, (img_size, round(img_size * a2)), interpolation = cv2.INTER_AREA)
        resized_img = r_img[0:img_size, 0:img_size]

    if(resized_img.shape[0] != img_size or resized_img.shape[1] != img_size):
        resized_img = r_img[0:img_size, 0:img_size]

    return resized_img

In [3]:
def print_color_values(img, shape_row=(0, 50), shape_col=(0, 60)):
    for row in range(shape_row[0], shape_row[1]):
        for col in range(shape_col[0], shape_col[1]):
            print(f"Row {row}: Spalte {col} Pixel: {img[row, col]}")

https://realpython.com/python-opencv-color-spaces/

The Code below is for data augmentation. It is a technique for enlarging a dataset.
It will use the standard techniques like horizontal & vertical flipping and rotation.

Load Images from personal cloud storage

In [6]:
path = "G:\Meine Ablage\Images_AI_Project\zwiebel_jpg\zwiebel_1.jpg"
onion = cv2.imread(path.replace("\\","/"))
#onion = cv2.cvtColor(onion, cv2.COLOR_BGR2RGB)

path = "G:\Meine Ablage\KI_Projekt\Bilder\Karotte\karotte_107.jpg"
carrot = cv2.imread(path.replace("\\","/"))
#carrot = cv2.cvtColor(carrot, cv2.COLOR_BGR2RGB)

path = "G:\Meine Ablage\KI_Projekt\Bilder\Karotte_Trieb\karotte_trieb_67.jpg"
carrot_trieb = cv2.imread(path.replace("\\","/"))
#carrot_trieb = cv2.cvtColor(carrot_trieb, cv2.COLOR_BGR2RGB)

path = "G:\Meine Ablage\Images_AI_Project\kartoffel_jpg\kartoffel_122.jpg"
potato = cv2.imread(path.replace("\\","/"))
#potato = cv2.cvtColor(potato, cv2.COLOR_BGR2RGB)

onion_sq = resize_to_square(onion)
carrot_sq = resize_to_square(carrot)
potato_sq = resize_to_square(potato)
carrot_trieb_sq = resize_to_square(carrot_trieb)

#show_image_plt(onion_sq)
#show_image_plt(carrot_sq)
#show_image_plt(potato_sq)
#show_image_plt(carrot_trieb_sq)

Applying the median filter https://theailearner.com/tag/cv2-medianblur/

https://machinelearningknowledge.ai/image-segmentation-in-python-opencv/

https://docs.opencv.org/4.x/de/d62/tutorial_bounding_rotated_ellipses.html

Approach: https://stackoverflow.com/questions/72100376/opencv-how-to-draw-a-rotated-bounding-box-in-python

compute dynamically thresholds: https://stackoverflow.com/questions/24862374/canny-edge-detector-threshold-values-gives-different-result

In [14]:
#path = "G:\Meine Ablage\KI_Projekt\Bilder\Zwiebel\zwiebel_9.jpg"
#vegi = cv2.imread(path.replace("\\","/")) # As BGR

def draw_contours(bgr_img, object_area=500):
    #gray = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)

    hue, saturation, value = cv2.split(hsv)
    
    blurred_sat = cv2.GaussianBlur(saturation, (7, 7), 0)
    #show_image_plt(blurred_sat)
    # Thresh 50 - 100
    # Or from 85 - 100 removes some more small edges
    # Thresh from 120 only shows the edge from vegi: karotte_107.jpg"
    # Compute the thresh dynamically from the mean() value. 
    thresh = blurred_sat.mean()
    # *****
    std = blurred_sat.std()
    thresh_low = thresh - std
    thresh_high = thresh + std
    # The factors were simply selected by testing the algoritm. Another approach could be to calculate the mean with the standard deviation -> Test it ???
    #thresh_low = 0.3 * thresh 
    #thresh_high = 2 * thresh
    
    #print(f"Thresh low: {thresh_low} and Thresh high: {thresh_high}")
    # The next four lines control how good the bounding box will fit
    edges = cv2.Canny(blurred_sat, thresh_low, thresh_high)
    #show_image_plt(edges)
    kernel = np.ones((4, 4), np.uint8) # creates 4x4 Identity matrix

    # To see the effect if changing kernel and iterations plot it. It seems if the value is to small than the probabillity is higher that the edge lines are not closed
    dilate = cv2.dilate(edges, kernel, iterations=4) 
    #show_image_plt(dilate)

    erode = cv2.erode(dilate, kernel, iterations=4)
    #show_image_plt(erode)
    #print("SHAPE: ", erode.shape)
    contours, hierarchy = cv2.findContours(erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10] # ???
   
    bgr_img_copy = bgr_img.copy()
    # Flag makes sure that there is a maximum of 1 box in each image. Assumption, the bounding box for the vegi is always the biggest
    more_than_one_box = False
    for i, contour in enumerate(contours):
        
        area = cv2.contourArea(contour)
        if area >= object_area:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(bgr_img_copy, [box], 0, (0, 255, 0), 2)
            
            # Ellipse
            #ellipse = cv2.fitEllipse(contour)
            #cv2.ellipse(bgr_img_copy, ellipse, (255, 255, 0), 2, cv2.LINE_AA)

            #perimeter = cv2.arcLength(contour, True)
            #circularity = (4 * np.pi * area) / (perimeter ** 2)
            #print(f"Perimeter: {perimeter}  /// Circularity: {circularity}")
            #print("Draw Contour")
            if i > 0:
                # There are more than 2 boxes in the image
                more_than_one_box = True
    
    rgb = cv2.cvtColor(bgr_img_copy, cv2.COLOR_BGR2RGB)
    
    return rgb, more_than_one_box, box, rect, area

def get_size_box(box):
    x0 = box[0][0]
    y0 = box[0][1]
    x1 = box[1][0]
    y1 = box[1][1]

    x2 = box[2][0]
    y2 = box[2][1]

    l0_1 = round(((x0 - x1)**2 + (y0 - y1)**2)**0.5, 2)
    l1_2 = round(((x1 - x2)**2 + (y1 - y2)**2)**0.5, 2)

    w = min(l0_1, l1_2)
    h = max(l0_1, l1_2)

    return h, w

def crop_minAreaRect(img_BGR, rect, h, w):
    pic = img_BGR.copy()
    # rotate img
    angle = rect[2]
    rows,cols = img_BGR.shape[0], img_BGR.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv2.warpAffine(img_BGR,M,(cols,rows))
    #show_image_plt(img_rot)
    
    # rotate bounding box
    rect0 = (rect[0], rect[1], 0.0) 
    box = cv2.boxPoints(rect0)
    box = np.intp(box)
    #print("BOX:", box)
    cv2.drawContours(img_rot, [box], 0, (255, 255, 255), 1) # white frame

    #pts = np.int0(cv2.transform(np.array([box]), M))[0]    
    #pts[pts < 0] = 0

    #print("TRANSFORMED BOX: ", pts)
    w_rect = int(rect[1][0])
    h_rect = int(rect[1][1])

    # crop
    box = np.clip(box, a_min=0, a_max=None)

    img_crop_BGR = img_rot[box[1][1]:box[1][1]+h_rect, box[1][0]:box[1][0]+w_rect]

    #img_crop = img_rot[pts[1][1]:pts[0][1], 
    #                   pts[1][0]:pts[2][0]]#

    return img_crop_BGR

def is_box_rotated(box):
    # If the box is not rotated then the top left corner should be the first element in box array
    x0, y0 = box[0][0], box[0][1]
    y1 = box[1][1]
    x3 = box[3][0]
    
    if y0 == y1 and x0 == x3:
        # box is not rotated
        return False
    # BOX IS ROTATED
    return True

def get_color(rgb_segment):
    cropped_vegi_2D = rgb_segment.reshape((-1,3))
    # convert to np.float32
    cropped_vegi_2D = np.float32(cropped_vegi_2D)

    # define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1)
    ret, label, center = cv2.kmeans(cropped_vegi_2D, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    center = np.uint8(center)
   
    res = center[label.flatten()]
    res2 = res.reshape((rgb_segment.shape))
    
    #returns center in rgb format
    return center, ret, label

def mask_green(cropped_vegi_seg_rgb, lower_thresh=(30, 175, 25), higher_thresh=(100, 255, 255)):
    ## Convert to HSV
    cropped_vegi_seg_hsv = cv2.cvtColor(cropped_vegi_seg_rgb, cv2.COLOR_RGB2HSV)

    ## Mask of green (36,25,25) ~ (86, 255,255)
    # mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))
    mask = cv2.inRange(cropped_vegi_seg_hsv, lower_thresh, higher_thresh)
    
    ## Slice the green
    imask = mask>0
    green_rgb = np.zeros_like(cropped_vegi_seg_rgb, np.uint8)
    green_rgb[imask] = cropped_vegi_seg_rgb[imask]

    return green_rgb, imask

    #Image as BGR
def segment_img_2(cropped_vegi_bgr):
    #img must be BGR
    gray = cv2.cvtColor(cropped_vegi_bgr, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU)


    #kernel = np.ones((3, 3), np.uint8)
    #closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE,kernel, iterations = 15)
    #bg = cv2.dilate(closing, kernel, iterations = 1)
    #dist_transform = cv2.distanceTransform(closing, cv2.DIST_L2, 0)
    #ret, fg = cv2.threshold(dist_transform, 0.02*dist_transform.max(), 255, 0)

    return thresh

def color_from_segmented_binary(seg_bin, cropped_vegi_bgr):
    imask = seg_bin>0 #False / True array
    segment = np.zeros_like(cropped_vegi_bgr, np.uint8)
    segment[imask] = cropped_vegi_bgr[imask] #BGR

    segment_rgb = cv2.cvtColor(segment, cv2.COLOR_BGR2RGB) #RGB

    return segment_rgb

def keep_object_remove_green(mask, cropped_vegi_seg_rgb):
    inverted_mask = np.invert(mask)
    inverted_green_rgb = np.zeros_like(cropped_vegi_seg_rgb, np.uint8)
    inverted_green_rgb[inverted_mask] = cropped_vegi_seg_rgb[inverted_mask]
    
    return inverted_green_rgb

def count_green_pixels(binary_green_mask):
    #only count True boolean. These are my green pixels
    return binary_green_mask.sum()
    


In [25]:
def debug_color(only_green_rgb): # binary mask
    #only_green_rgb =  cv2.cvtColor(only_green, cv2.COLOR_GRAY2RGB)
    only_green_hsv =  cv2.cvtColor(only_green_rgb, cv2.COLOR_RGB2HSV)
    plt.imshow(only_green_hsv)
    plt.show()

    h, s, v = cv2.split(only_green_hsv)
    h_1d = h.flatten()
    s_1d = s.flatten()
    v_1d = v.flatten()

    h_mean_counter = 0
    v_mean_counter = 0
    s_mean_counter = 0
    sum_h = 0
    sum_v = 0
    sum_s = 0
    for i in range(0, h_1d.shape[0]):
        if h_1d[i] > 1:
            sum_h += h_1d[i]
            h_mean_counter += 1

        if v_1d[i] > 1:
            sum_v += v_1d[i]
            v_mean_counter += 1

        if s_1d[i] > 1:
            sum_s += s_1d[i]
            s_mean_counter += 1

    mean_h = int(sum_h / h_mean_counter)
    mean_v = int(sum_v / v_mean_counter)
    mean_s = int(sum_s / s_mean_counter)

    print(f"H: {mean_h} / S: {mean_s} / V: {mean_v} ")
    print(h_mean_counter, s_mean_counter, v_mean_counter)

    hsv = [mean_h, mean_s, mean_v]
    rgb = cv2.cvtColor( np.uint8([[hsv]] ), cv2.COLOR_HSV2RGB)[0][0]
    print("RGB: ", rgb)

def segment_potato_tribes(cropped_bgr):
    cropped_gray = cv2.cvtColor(cropped_bgr, cv2.COLOR_BGR2GRAY)
    #print("Cropped Gray")
    #show_image_plt(cropped_gray)
    thresh_low = 100
    thresh_high = 150
    edges = cv2.Canny(cropped_gray, thresh_low, thresh_high)
    print("Edges Gray")
    show_image_plt(edges)

    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    gray_img_copy = cropped_gray.copy()
    # Flag makes sure that there is a maximum of 1 box in each image. Assumption, the bounding box for the vegi is always the biggest
    more_than_one_box = False
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
     
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.intp(box)
        cv2.drawContours(gray_img_copy, [box], 0, (0, 255, 0), 1)
    
    plt.imshow(gray_img_copy, cmap="gray")
    plt.show()
    

Load images from storage and pass them through the preprocessing pipeline. The individual steps are:<br>
- Read Image as BGR Format<br>
- Resize Image to square format<br>
- Draw bounding box around the object in the image and convert it to RGB<br>
- get Hight and Width from the bounding box<br>
- check If the box is rotated and depending on that crop the area of the bounding box out of the image<br>
- Do Object Segmentation on the cropped image and return the binary segmented img (mask)<br>
- Get the segmented cropped RGB image<br>
- Place a green mask on the segmented cropped RGB image and get only the green pixels<br>
- Calculate the color from the segmented cropped RGB image with kmeans algorithm. Set k=2<br>
- Write the desired features to the dataframe

In [9]:
paths = []
for fold in folders:
    lower = fold.lower()
    for i in range(100, 101): #87, 91
        paths.append(f"G:/Meine Ablage/KI_Projekt/Bilder/{fold}/{lower}_{i}.jpg")
a = paths[0].split(".")
b = a[0].split("_")
print(b[-1])
print(b[-2])
if b[]
print(b[-3].replace("/", "_"))

SyntaxError: invalid syntax (1118774210.py, line 10)

In [29]:
folders = ["Kartoffel", "Karotte", "Zwiebel", "Karotte_Trieb", "Zwiebel_Trieb", "Kartoffel_Trieb"]
paths = []
for fold in folders:
    for fname in os.listdir(f"G:/Meine Ablage/KI_Projekt/Bilder/Bilder_LGR/{fold}"):
        paths.append(f"G:/Meine Ablage/KI_Projekt/Bilder/Bilder_LGR/{fold}/{fname}")
        

In [36]:
paths[331]

'G:/Meine Ablage/KI_Projekt/Bilder/Bilder_LGR/Kartoffel/desktop.ini'

In [38]:
t = 'G:/Meine Ablage/KI_Projekt/Bilder/Bilder_LGR/Kartoffel/desktop.ini'
if 'desktop.ini' in t:
    print("yes")
else:
    print("NO")

NO


In [39]:
#paths = [f"G:\Meine Ablage\KI_Projekt\Bilder\Zwiebel\zwiebel_{i}.jpg" for i in range(10, 133)]
#paths = [f"G:\Meine Ablage\Images_AI_Project\kartoffel_jpg_2\kartoffel_{i}.jpg" for i in range(40, 50)]

#folders = ["Kartoffel", "Karotte", "Zwiebel", "Karotte_Trieb", "Zwiebel_Trieb", "Kartoffel_Trieb"]
#folders = ["Karotte", "Zwiebel"]
#folders = ["Kartoffel_Trieb", "Karotte"]

#for fold in folders:
#    lower = fold.lower()
#    for i in range(155, 159): #87, 91
#        paths.append(f"G:/Meine Ablage/KI_Projekt/Bilder/Bilder_LGR/{fold}/{lower}_{i}.jpg")

vegis_df = pd.DataFrame(columns=["Hight", "Width", "R", "G", "B", "green pixels", "green onion pixels", "ratio area", "Label", "Image"])
debug_list = list()        
center_points = np.array([0, 0, 0])
for i, path in enumerate(paths):
    #print(f"++++++++++++++++++++NEW IMAGE +++++++++++++++++++++++\n")
    if 'desktop.ini' in path:
        continue
    vegi_label = path.split("/")[5]
 
    #vegi_BGR = cv2.imread(path.replace("\\","/")) # As BGR
    vegi_BGR = cv2.imread(path) # As BGR
    vegi_sq_BGR_original = resize_to_square(vegi_BGR) # BGR
    #show_image_plt(vegi_sq)
    try:
        vegi_with_box_rgb, more_than_one_box, box, rect, area = draw_contours(vegi_sq_BGR_original)
        if more_than_one_box:
            box_size = 500
            while more_than_one_box:
                #print("Increase Box Size by 250")
                box_size += 250
                vegi_with_box_rgb, more_than_one_box, box, rect, area = draw_contours(vegi_sq_BGR_original, box_size)

        #show_image_plt(vegi_with_box_rgb)
        h, w = [int(l) for l in get_size_box(box)]
        #print(f"Height: {h}. width: {w}")
        ratio_area_box = 0
        if h > 0 and w > 0:
            ratio_area_box = area / (h * w)

        if not is_box_rotated(box):
            #crop img directly
            #print("CROP DIRECTLY")
            # the top left corner should be the first element in box array
            cropped_vegi_BGR = vegi_sq_BGR_original[box[0][1]:box[0][1]+h, box[0][0]:box[0][0]+w]
            
        else:
            cropped_vegi_BGR = crop_minAreaRect(vegi_sq_BGR_original, rect, h, w) #BGR
    
        cropped_segmented_binary = segment_img_2(cropped_vegi_BGR)
        #print("Cropped Vegi BGR")
        #show_image_plt(cropped_vegi_BGR)
        
        #print("Segment whole object")
        cropped_vegi_segmented_rgb = color_from_segmented_binary(cropped_segmented_binary, cropped_vegi_BGR) # center has rgb format
        #show_image_plt(cropped_vegi_segmented_rgb)
        ############################# MASK GREEN CARROT TRIBES ##################################
        #lower_thresh=(30, 105, 20), higher_thresh=(160, 255, 255) good default values.
        vegi_only_green_rgb, green_mask = mask_green(cropped_vegi_segmented_rgb, lower_thresh=(27, 105, 20), higher_thresh=(120, 255, 255))
        #print("ONLY GREEN MASK")
        #show_image_plt(vegi_only_green_rgb)
        ############################# END ##################################

        ############################# MASK GREEN POTATO TRIBES ##################################
        # Seems to be work best
        vegi_only_onion_tribe_rgb, onion_tribe_mask = mask_green(cropped_vegi_segmented_rgb, lower_thresh=(25, 40, 20), higher_thresh=(60, 160, 255))
        #print("ONLY GREEn TRIBE MASK")
        #show_image_plt(vegi_only_onion_tribe_rgb)
        ############################# END ##################################
        
        # Keep object without
        cropped_vegi_green_removed_rgb = keep_object_remove_green(green_mask, cropped_vegi_segmented_rgb)

        amount_green_pixels = count_green_pixels(green_mask)
        amount_green_pixels_onion_tribe = count_green_pixels(onion_tribe_mask)
        #print("AMOUNT OF GREEN;: ", amount_green_pixels)

        center, ret, label = get_color(cropped_vegi_segmented_rgb) #Center = RGB
        R, G, B = np.max(center, axis=0)
        #print("Center K-Means Algorith: ", center)
        #print("Center MAX: ", R, G, B)

        #rgb_mean_color = calculate_object_color(cropped_vegi_green_removed_rgb)
        #print(f"Mean RGB COLOR: {rgb_mean_color}\n\n\n")

        #center_points = np.vstack([center_points, rgb_mean_color])

        ########################################################################################
        #New Approach tribe detection onion - conv to gray


        ########################################################################################


        #debug_list.append(amount_green_pixels)
        a = path.split(".")
        b = a[0].split("_")
        img_name = f"{vegi_label}_{b[-1]}"
        vegis_df.loc[len(vegis_df)] = [h, w, R, G, B, amount_green_pixels, amount_green_pixels_onion_tribe, ratio_area_box, vegi_label, img_name]
        
        if i < 3:
            print(vegis_df)
        #debug_color(vegi_only_green_rgb)
        
        #cv2.imwrite(f"G:\Meine Ablage\KI_Projekt\Bilder\BildermitBounding\green_mask_test\only_green_carrot\{vegi_label}_{b[-1]}.jpg", cv2.cvtColor(vegi_only_green_rgb, cv2.COLOR_BGR2RGB))
        #cv2.imwrite(f"G:\Meine Ablage\KI_Projekt\Bilder\BildermitBounding\green_mask_test\only_green_onion\{vegi_label}_{b[-1]}.jpg", cv2.cvtColor(vegi_only_onion_tribe_rgb, cv2.COLOR_BGR2RGB))
        #cv2.imwrite(f"G:\Meine Ablage\KI_Projekt\Bilder\BildermitBounding\green_mask_test\segmented_rgb\{vegi_label}_{b[-1]}.jpg", cv2.cvtColor(cropped_vegi_segmented_rgb, cv2.COLOR_BGR2RGB))
    except Exception as e:
        print(f"############# Error: {e}")

   Hight  Width    R   G   B  green pixels  green onion pixels  ratio area  \
0    131     97  131  98  59             0                   0    0.805619   

       Label        Image  
0  Kartoffel  Kartoffel_0  
   Hight  Width    R    G   B  green pixels  green onion pixels  ratio area  \
0    131     97  131   98  59             0                   0    0.805619   
1    128    111  141  106  61             0                   0    0.796523   

       Label        Image  
0  Kartoffel  Kartoffel_0  
1  Kartoffel  Kartoffel_1  
   Hight  Width    R    G   B  green pixels  green onion pixels  ratio area  \
0    131     97  131   98  59             0                   0    0.805619   
1    128    111  141  106  61             0                   0    0.796523   
2    152    116  148  118  75             0                   0    0.787971   

       Label        Image  
0  Kartoffel  Kartoffel_0  
1  Kartoffel  Kartoffel_1  
2  Kartoffel  Kartoffel_2  
############# Error: local variable 

In [40]:
vegis_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1177 entries, 0 to 1176
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hight               1177 non-null   int64  
 1   Width               1177 non-null   int64  
 2   R                   1177 non-null   uint8  
 3   G                   1177 non-null   uint8  
 4   B                   1177 non-null   uint8  
 5   green pixels        1177 non-null   int32  
 6   green onion pixels  1177 non-null   int32  
 7   ratio area          1177 non-null   float64
 8   Label               1177 non-null   object 
 9   Image               1177 non-null   object 
dtypes: float64(1), int32(2), int64(2), object(2), uint8(3)
memory usage: 67.8+ KB


In [42]:
vegis_df.head(15)

,Hight,Width,R,G,B,green pixels,green onion pixels,ratio area,Label,Image
0,131,97,131,98,59,0,0,0.805619,Kartoffel,Kartoffel_0
1,128,111,141,106,61,0,0,0.796523,Kartoffel,Kartoffel_1
2,152,116,148,118,75,0,0,0.787971,Kartoffel,Kartoffel_2
3,111,108,153,122,76,0,0,0.794086,Kartoffel,Kartoffel_3
4,131,114,144,112,68,0,0,0.037900,Kartoffel,Kartoffel_4
5,220,136,155,125,78,0,0,0.780966,Kartoffel,Kartoffel_5
6,180,116,152,120,72,0,0,0.787716,Kartoffel,Kartoffel_6
7,177,116,143,111,68,0,0,0.792105,Kartoffel,Kartoffel_7
8,209,149,126,94,55,0,0,0.308677,Kartoffel,Kartoffel_8
9,195,137,127,96,59,0,0,0.792121,Kartoffel,Kartoffel_9


In [43]:
vegis_df.to_csv("G:/Meine Ablage/KI_Projekt/Daten/vegi_benchmark.csv")

In [41]:
vegis_df.head()
vegis_df.shape

(1177, 10)